In [31]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import glob
import csv
import pandas as pd
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")

In [20]:
base_folder_location = '../scripts/Final/'

autoencoder_location = os.path.join(base_folder_location, 'Autoencoder/experiments/*')
autoencoder_fixmatch_location = os.path.join(base_folder_location, 'AutoencoderFixMatch/experiments/*')
fixmatch_location = os.path.join(base_folder_location, 'FixMatch/experiments/*')
mixmatch_location = os.path.join(base_folder_location, 'MixMatch/experiments/*')

In [98]:
def get_test_information(location, columns_names, columns_file_values):
    all_columns = []
    all_columns.extend(columns_names)
    all_columns.extend(columns_file_values)

    df = pd.DataFrame(columns=all_columns)
    
    for experiment in glob.glob(location):
        folder_name = os.path.basename(experiment)
        hyper_parameters = folder_name.split('_')
        
        test_summary_location = os.path.join(experiment, 'result_outputs/test_summary.csv')
        
        with open(test_summary_location, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)
            values = next(csvreader)
            
            all_values = []
            all_values.extend(hyper_parameters)
            all_values.extend(values)
            
            df = df.append(dict(zip(df.columns, all_values)), ignore_index=True)
    
    return df

df_mixmatch = get_test_information(mixmatch_location, ['Model', 'Seed', 'Magnification', 'Labeled Amount'], ['Test Acc', 'Test Loss'])

df_mixmatch["Test Loss"] = pd.to_numeric(df_mixmatch["Test Loss"])
df_mixmatch["Labeled Amount"] = pd.to_numeric(df_mixmatch["Labeled Amount"])
df_mixmatch["Test Acc"] = pd.to_numeric(df_mixmatch["Test Acc"])

df_fixmatch = get_test_information(fixmatch_location, ['Model', 'Seed', 'Magnification', 'Labeled Amount'], ['Test Acc', 'Test Loss'])

df_fixmatch["Test Loss"] = pd.to_numeric(df_fixmatch["Test Loss"])
df_fixmatch["Labeled Amount"] = pd.to_numeric(df_fixmatch["Labeled Amount"])
df_fixmatch["Test Acc"] = pd.to_numeric(df_fixmatch["Test Acc"])

df_autofixmatch = get_test_information(autoencoder_fixmatch_location, ['Model', 'Seed', 'Magnification', 'Labeled Amount'], ['Test Acc', 'Test Loss'])

df_autofixmatch["Test Loss"] = pd.to_numeric(df_autofixmatch["Test Loss"])
df_autofixmatch["Labeled Amount"] = pd.to_numeric(df_autofixmatch["Labeled Amount"])
df_autofixmatch["Test Acc"] = pd.to_numeric(df_autofixmatch["Test Acc"])

In [101]:
df_all = df_mixmatch.append(df_fixmatch).append(df_autofixmatch)


In [103]:
for magnification in ['40X', '100X', '200X', '400X']:
    df = df_all[df_all['Magnification'] == magnification]

    sns.lineplot(x="Labeled Amount", y="Test Loss", 
                 hue="Model", style="Model",
                 markers=True, data=df)
    
    plt.savefig(f'{magnification}.pdf')
    plt.clf()


<Figure size 432x288 with 0 Axes>